# Paper 14: Neural Machine Translation by Jointly Learning to Align and Translate
## Dzmitry Bahdanau, KyungHyun Cho, Yoshua Bengio (2014)

### The Original Attention Mechanism

This paper introduced **attention** - one of the most important innovations in deep learning. It preceded Transformers by 3 years!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## The Problem: Fixed-Length Context Vector

Traditional seq2seq compresses entire input into single vector → information bottleneck!

In [ ]:
def softmax(x, axis=-1):
    exp_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

class EncoderRNN:
    """Bidirectional RNN encoder"""
    def __init__(self, input_size, hidden_size):
        self.hidden_size = hidden_size
        
        # Forward RNN
        self.W_fwd = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_fwd = np.zeros((hidden_size, 1))
        
        # Backward RNN
        self.W_bwd = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_bwd = np.zeros((hidden_size, 1))
    
    def forward(self, inputs):
        """
        inputs: list of (input_size, 1) vectors
        Returns: list of bidirectional hidden states (2*hidden_size, 1)
        """
        seq_len = len(inputs)
        
        # Forward pass
        h_fwd = []
        h = np.zeros((self.hidden_size, 1))
        for x in inputs:
            concat = np.vstack([x, h])
            h = np.tanh(np.dot(self.W_fwd, concat) + self.b_fwd)
            h_fwd.append(h)
        
        # Backward pass
        h_bwd = []
        h = np.zeros((self.hidden_size, 1))
        for x in reversed(inputs):
            concat = np.vstack([x, h])
            h = np.tanh(np.dot(self.W_bwd, concat) + self.b_bwd)
            h_bwd.append(h)
        h_bwd = list(reversed(h_bwd))
        
        # Concatenate forward and backward
        annotations = [np.vstack([h_f, h_b]) for h_f, h_b in zip(h_fwd, h_bwd)]
        
        return annotations

print("Bidirectional Encoder created")

## Bahdanau Attention Mechanism

The key innovation: align and translate jointly!

**Attention score**: $e_{ij} = a(s_{i-1}, h_j)$ where $s$ is decoder state, $h$ is encoder annotation

**Attention weights**: $\alpha_{ij} = \frac{\exp(e_{ij})}{\sum_k \exp(e_{ik})}$

**Context vector**: $c_i = \sum_j \alpha_{ij} h_j$

In [ ]:
class BahdanauAttention:
    """Additive attention mechanism"""
    def __init__(self, hidden_size, annotation_size):
        self.hidden_size = hidden_size
        
        # Attention parameters
        self.W_a = np.random.randn(hidden_size, hidden_size) * 0.01
        self.U_a = np.random.randn(hidden_size, annotation_size) * 0.01
        self.v_a = np.random.randn(1, hidden_size) * 0.01
    
    def forward(self, decoder_hidden, encoder_annotations):
        """
        decoder_hidden: (hidden_size, 1) - current decoder state s_{i-1}
        encoder_annotations: list of (annotation_size, 1) - all encoder states h_j
        
        Returns:
        context: (annotation_size, 1) - weighted sum of annotations
        attention_weights: (seq_len,) - attention distribution
        """
        scores = []
        
        # Compute attention scores for each position
        for h_j in encoder_annotations:
            # e_ij = v_a^T * tanh(W_a * s_{i-1} + U_a * h_j)
            score = np.dot(self.v_a, np.tanh(
                np.dot(self.W_a, decoder_hidden) + 
                np.dot(self.U_a, h_j)
            ))
            scores.append(score[0, 0])
        
        # Softmax to get attention weights
        scores = np.array(scores)
        attention_weights = softmax(scores)
        
        # Compute context vector as weighted sum
        context = sum(alpha * h for alpha, h in zip(attention_weights, encoder_annotations))
        
        return context, attention_weights

print("Bahdanau Attention mechanism created")

## Decoder with Attention

In [ ]:
class AttentionDecoder:
    """RNN decoder with Bahdanau attention"""
    def __init__(self, output_size, hidden_size, annotation_size):
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Attention mechanism
        self.attention = BahdanauAttention(hidden_size, annotation_size)
        
        # RNN: takes previous output + context
        input_size = output_size + annotation_size
        self.W_dec = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_dec = np.zeros((hidden_size, 1))
        
        # Output layer
        self.W_out = np.random.randn(output_size, hidden_size + annotation_size + output_size) * 0.01
        self.b_out = np.zeros((output_size, 1))
    
    def step(self, prev_output, decoder_hidden, encoder_annotations):
        """
        Single decoding step
        
        prev_output: (output_size, 1) - previous output word
        decoder_hidden: (hidden_size, 1) - previous decoder state
        encoder_annotations: list of (annotation_size, 1) - encoder states
        
        Returns:
        output: (output_size, 1) - predicted output distribution
        new_hidden: (hidden_size, 1) - new decoder state
        attention_weights: attention distribution
        """
        # Compute attention and context
        context, attention_weights = self.attention.forward(decoder_hidden, encoder_annotations)
        
        # Decoder RNN: s_i = f(s_{i-1}, y_{i-1}, c_i)
        rnn_input = np.vstack([prev_output, context])
        concat = np.vstack([rnn_input, decoder_hidden])
        new_hidden = np.tanh(np.dot(self.W_dec, concat) + self.b_dec)
        
        # Output: y_i = g(s_i, y_{i-1}, c_i)
        output_input = np.vstack([new_hidden, context, prev_output])
        output = np.dot(self.W_out, output_input) + self.b_out
        
        return output, new_hidden, attention_weights
    
    def forward(self, encoder_annotations, max_length=20, start_token=None):
        """
        Full decoding
        """
        if start_token is None:
            start_token = np.zeros((self.output_size, 1))
        
        outputs = []
        attention_history = []
        
        # Initialize
        decoder_hidden = np.zeros((self.hidden_size, 1))
        prev_output = start_token
        
        for _ in range(max_length):
            output, decoder_hidden, attention_weights = self.step(
                prev_output, decoder_hidden, encoder_annotations
            )
            
            outputs.append(output)
            attention_history.append(attention_weights)
            
            # Next input is current output (greedy decoding)
            prev_output = output
        
        return outputs, attention_history

print("Attention Decoder created")

## Complete Seq2Seq with Attention

In [ ]:
class Seq2SeqWithAttention:
    def __init__(self, input_vocab_size, output_vocab_size, hidden_size=32):
        self.input_vocab_size = input_vocab_size
        self.output_vocab_size = output_vocab_size
        self.hidden_size = hidden_size
        
        # Embedding layers
        self.input_embedding = np.random.randn(input_vocab_size, hidden_size) * 0.01
        self.output_embedding = np.random.randn(output_vocab_size, hidden_size) * 0.01
        
        # Encoder (bidirectional, so annotation size is 2*hidden_size)
        self.encoder = EncoderRNN(hidden_size, hidden_size)
        
        # Decoder with attention
        annotation_size = 2 * hidden_size
        self.decoder = AttentionDecoder(hidden_size, hidden_size, annotation_size)
    
    def translate(self, input_sequence, max_output_length=15):
        """
        Translate input sequence to output sequence
        
        input_sequence: list of token indices
        """
        # Embed input
        embedded = [self.input_embedding[idx:idx+1].T for idx in input_sequence]
        
        # Encode
        annotations = self.encoder.forward(embedded)
        
        # Decode
        start_token = self.output_embedding[0:1].T  # Use first token as start
        outputs, attention_history = self.decoder.forward(
            annotations, max_length=max_output_length, start_token=start_token
        )
        
        return outputs, attention_history, annotations

# Create model
input_vocab_size = 20   # Source language vocab
output_vocab_size = 20  # Target language vocab
model = Seq2SeqWithAttention(input_vocab_size, output_vocab_size, hidden_size=16)

print(f"Seq2Seq with Attention created")
print(f"Input vocab: {input_vocab_size}")
print(f"Output vocab: {output_vocab_size}")

## Test on Synthetic Translation Task

In [ ]:
# Simple synthetic task: reverse sequence
# Input: [1, 2, 3, 4, 5]
# Output: [5, 4, 3, 2, 1]

input_seq = [1, 2, 3, 4, 5, 6, 7]
outputs, attention_history, annotations = model.translate(input_seq, max_output_length=len(input_seq))

print(f"Input sequence: {input_seq}")
print(f"Number of output steps: {len(outputs)}")
print(f"Number of attention distributions: {len(attention_history)}")
print(f"Encoder annotations shape: {len(annotations)} x {annotations[0].shape}")

## Visualize Attention Weights

The key insight: see what the model attends to!

In [ ]:
# Convert attention history to matrix
attention_matrix = np.array(attention_history)  # (output_len, input_len)

plt.figure(figsize=(10, 8))
plt.imshow(attention_matrix, cmap='Blues', aspect='auto', interpolation='nearest')
plt.colorbar(label='Attention Weight')
plt.xlabel('Input Position (Source)')
plt.ylabel('Output Position (Target)')
plt.title('Bahdanau Attention Alignment Matrix')

# Add grid
plt.xticks(range(len(input_seq)), [f'x{i+1}' for i in input_seq])
plt.yticks(range(len(outputs)), [f'y{i+1}' for i in range(len(outputs))])

plt.tight_layout()
plt.show()

print("\nAttention patterns show which input positions influence each output.")
print("Brighter cells = higher attention weight.")

## Attention at Each Decoder Step

In [ ]:
# Visualize attention distribution at specific decoder steps
fig, axes = plt.subplots(2, 4, figsize=(16, 6))
axes = axes.flatten()

steps_to_show = min(8, len(attention_history))

for i in range(steps_to_show):
    axes[i].bar(range(len(input_seq)), attention_history[i])
    axes[i].set_title(f'Output Step {i+1}')
    axes[i].set_xlabel('Input Position')
    axes[i].set_ylabel('Attention Weight')
    axes[i].set_ylim(0, 1)
    axes[i].set_xticks(range(len(input_seq)))
    axes[i].set_xticklabels([f'x{j+1}' for j in input_seq], fontsize=8)
    axes[i].grid(True, alpha=0.3, axis='y')

plt.suptitle('Attention Distribution at Each Decoding Step', fontsize=14)
plt.tight_layout()
plt.show()

print("Each decoder step focuses on different input positions!")

## Compare: With vs Without Attention

In [ ]:
# Simulate fixed-context seq2seq (no attention)
def fixed_context_attention(seq_len):
    """Simulates attending only to last encoder state"""
    weights = np.zeros(seq_len)
    weights[-1] = 1.0  # Only attend to last position
    return weights

# Create comparison
input_length = len(input_seq)
output_length = len(outputs)

# Fixed context
fixed_attention = np.array([fixed_context_attention(input_length) for _ in range(output_length)])

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Without attention (fixed context)
im1 = ax1.imshow(fixed_attention, cmap='Blues', aspect='auto', vmin=0, vmax=1)
ax1.set_xlabel('Input Position')
ax1.set_ylabel('Output Position')
ax1.set_title('Without Attention (Fixed Context)\nAll decoder steps see only last encoder state')
plt.colorbar(im1, ax=ax1)

# With Bahdanau attention
im2 = ax2.imshow(attention_matrix, cmap='Blues', aspect='auto', vmin=0, vmax=1)
ax2.set_xlabel('Input Position')
ax2.set_ylabel('Output Position')
ax2.set_title('With Bahdanau Attention\nEach decoder step attends to different positions')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

print("\nKey Difference:")
print("  Without attention: Information bottleneck at last encoder state")
print("  With attention: Dynamic access to all encoder states")

## Attention Mechanism Variants

In [ ]:
def bahdanau_score(s, h, W_a, U_a, v_a):
    """Additive/Concat attention (Bahdanau)"""
    return np.dot(v_a.T, np.tanh(np.dot(W_a, s) + np.dot(U_a, h)))[0, 0]

def dot_product_score(s, h):
    """Dot product attention (Luong)"""
    return np.dot(s.T, h)[0, 0]

def scaled_dot_product_score(s, h):
    """Scaled dot product (Transformer-style)"""
    d_k = s.shape[0]
    return np.dot(s.T, h)[0, 0] / np.sqrt(d_k)

# Compare scoring functions
s = np.random.randn(16, 1)
h = np.random.randn(32, 1)
W_a = np.random.randn(16, 16)
U_a = np.random.randn(16, 32)
v_a = np.random.randn(1, 16)

print("Attention Score Functions:")
print(f"  Bahdanau (additive): score = v^T tanh(W*s + U*h)")
print(f"  Dot product: score = s^T h")
print(f"  Scaled dot product: score = s^T h / sqrt(d_k)")
print(f"\nBahdanau is more expressive but has more parameters.")

## Key Takeaways

### The Problem Attention Solves:
- **Fixed-length context**: Entire input compressed to single vector
- **Information bottleneck**: Long sequences lose information
- **No alignment**: Decoder doesn't know which input to focus on

### Bahdanau Attention Innovation:
1. **Dynamic context**: Different for each decoder step
2. **Soft alignment**: Learns to align source and target
3. **All encoder states**: Decoder has access to all, not just last

### How It Works:
```
1. Encoder produces annotations h_1, ..., h_T
2. For each decoder step i:
   a. Compute attention scores: e_ij = score(s_{i-1}, h_j)
   b. Normalize to weights: α_ij = softmax(e_ij)
   c. Compute context: c_i = Σ α_ij * h_j
   d. Generate output: y_i = f(s_i, c_i, y_{i-1})
```

### Bahdanau vs Luong Attention:
| Feature | Bahdanau (2014) | Luong (2015) |
|---------|----------------|---------------|
| Score | Additive: v·tanh(W·s + U·h) | Multiplicative: s·h |
| When | Uses s_{i-1} (previous) | Uses s_i (current) |
| Global/Local | Global only | Both options |

### Mathematical Formulation:

**Attention score (alignment model)**:
$$e_{ij} = v_a^T \tanh(W_a s_{i-1} + U_a h_j)$$

**Attention weights**:
$$\alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})}$$

**Context vector**:
$$c_i = \sum_{j=1}^{T_x} \alpha_{ij} h_j$$

**Decoder**:
$$s_i = f(s_{i-1}, y_{i-1}, c_i)$$
$$p(y_i | y_{<i}, x) = g(s_i, y_{i-1}, c_i)$$

### Impact:
- **Revolutionized NMT**: BLEU scores jumped significantly
- **Interpretability**: Can visualize alignments
- **Foundation for Transformers**: Pure attention (2017)
- **Beyond NMT**: Used in vision, speech, etc.

### Why It Worked:
1. **Solves bottleneck**: Variable-length context
2. **Learns alignment**: No need for separate alignment model
3. **Differentiable**: End-to-end training
4. **Works for long sequences**: Attention doesn't decay

### Modern Perspective:
- Transformers use **self-attention** (attend to same sequence)
- Scaled dot-product is now standard (simpler, faster)
- Multi-head attention captures different relationships
- But Bahdanau's core idea remains: **attend to what's relevant**